In [105]:
import pandas as pd
import glob
import numpy as np
import shutil

In [106]:
#test file path
train_file = "./dataset/train"
#input folder which needs to be segregated
input_path = "./dataset/test" 

In [107]:
########TRAINING DATASET PREPARATION#############################

file_path = train_file
for files in glob.glob(file_path + "/*", recursive=True):

    print(files)

dataset = pd.DataFrame()
list_file = []
type_file = []
content_file = []

for f in glob.glob(file_path + "/*", recursive=True):

    for files in glob.glob(f + "/*" + "." + "txt", recursive=True):
        file = open(files, 'r')
        content = file.read()

        list_file.append(files)
        type_file.append(f.split('\\')[-1])
        content_file.append(content)

dataset['File Names'] = list_file
dataset['Category'] = type_file
dataset['Content'] = content_file
dataset['category_id'] = dataset['Category'].factorize()[0]

category_id_dataset = dataset[['Category', 'category_id']].drop_duplicates().sort_values('category_id')

dataset['Category'].value_counts()

./dataset/train/entertainment
./dataset/train/politics
./dataset/train/business
./dataset/train/sport
./dataset/train/tech


./dataset/train/tech             10
./dataset/train/entertainment    10
./dataset/train/business         10
./dataset/train/sport            10
./dataset/train/politics         10
Name: Category, dtype: int64

In [108]:
dataset

,File Names,Category,Content,category_id
0,./dataset/train/entertainment/021.txt,./dataset/train/entertainment,Obituary: Dame Alicia Markova\n\nDame Alicia M...,0
1,./dataset/train/entertainment/035.txt,./dataset/train/entertainment,Hundreds vie for best film Oscar\n\nA total of...,0
2,./dataset/train/entertainment/043.txt,./dataset/train/entertainment,Stars pay tribute to actor Davis\n\nHollywood ...,0
3,./dataset/train/entertainment/049.txt,./dataset/train/entertainment,De Niro film leads US box office\n\nFilm star ...,0
4,./dataset/train/entertainment/023.txt,./dataset/train/entertainment,Famed music director Viotti dies\n\nConductor ...,0
5,./dataset/train/entertainment/042.txt,./dataset/train/entertainment,Berlin honours S Korean director\n\nSouth Kore...,0
6,./dataset/train/entertainment/039.txt,./dataset/train/entertainment,Aviator 'creator' in Oscars snub\n\nThe man wh...,0
7,./dataset/train/entertainment/022.txt,./dataset/train/entertainment,Fears raised over ballet future\n\nFewer child...,0
8,./dataset/train/entertainment/048.txt,./dataset/train/entertainment,'Landmark movies' of 2004 hailed\n\nUS film pr...,0
9,./dataset/train/entertainment/050.txt,./dataset/train/entertainment,Willis sues over movie 'injury'\n\nActor Bruce...,0


In [109]:
#####################TF-IDF, FEATURE EXTRACTION AND LABEL######################

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df = 2, norm='l2', encoding='latin-1', 
                        ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(dataset.Content).toarray()
labels = dataset.category_id
features.shape

(50, 1543)

In [110]:
features

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04325114, 0.        , 0.03961084, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [111]:
########################N-GRAMS################################################

from sklearn.feature_selection import chi2

category_to_id = dict(category_id_dataset.values)
id_to_category = dict(category_id_dataset[['category_id', 'Category']].values)
N = 5
for category, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
    print("# '{}':".format(category))
    print("  . Most correlated unigrams:\n       . {}".format('\n       . '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n       . {}".format('\n       . '.join(bigrams[-N:])))
    print("  . Most correlated trigrams:\n       . {}".format('\n       . '.join(trigrams[-N:])))

# './dataset/train/business':
  . Most correlated unigrams:
       . economy
       . friday
       . unit
       . yukos
       . growth
  . Most correlated bigrams:
       . economic growth
       . government hopes
       . year march
       . march 2005
       . g7 meeting
  . Most correlated trigrams:
       . oil giant yukos
       . forecast year march
       . year march 2005
       . major cost cutting
       . longer term view
# './dataset/train/entertainment':
  . Most correlated unigrams:
       . aviator
       . oscar
       . opera
       . ballet
       . film
  . Most correlated bigrams:
       . aviator million
       . box office
       . best film
       . dollar baby
       . million dollar
  . Most correlated trigrams:
       . director martin scorsese
       . dollar baby sideways
       . best film oscar
       . aviator million dollar
       . million dollar baby
# './dataset/train/politics':
  . Most correlated unigrams:
       . tory
       . public
       . 

In [112]:
###########################Logistic Regression MODEL TRAINING######################################

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

logistic_regression = LogisticRegression(random_state=1)

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, 
                                                                                 dataset.index, 
                                                                                 test_size=2, 
                                                                                 random_state=1)
logistic_regression.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
#################################RANDOM FOREST CLASSIFIER TRAINING#############

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=1)

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, 
                                                                                 dataset.index, 
                                                                                 test_size=2, 
                                                                                 random_state=1)
random_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [114]:
###########################MULTIONOMIAL NAIVE BAYES MODEL TRAINING#############

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

multinomial_naive_bayes = MultinomialNB()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, 
                                                                                 dataset.index, 
                                                                                 test_size=2, 
                                                                                 random_state=7)
multinomial_naive_bayes.fit(X_train, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [115]:
##############################TEST DATASET PREPARATION#########################

test_file = input_path

dataset_test = pd.DataFrame()
list_file_test = []
content_file_test = []

for files in glob.glob(test_file + "/*" + "." + "txt", recursive=True):
    file = open(files, 'r')
    content = file.read()

    list_file_test.append(files)
    content_file_test.append(content)

dataset_test['File Names'] = list_file_test
dataset_test['Content'] = content_file_test

In [116]:
dataset_test

,File Names,Content
0,./dataset/test/politics(2).txt,"Howard's unfinished business\n\n""He's not fini..."
1,./dataset/test/tech (8).txt,'Brainwave' cap controls computer\n\nA team of...
2,./dataset/test/tech (15).txt,What price for 'trusted PC security'?\n\nYou c...
3,./dataset/test/tech (16).txt,Text message record smashed again\n\nUK mobile...
4,./dataset/test/business(3).txt,Worldcom boss 'left books alone'\n\nFormer Wor...
5,./dataset/test/tech (6).txt,Looks and music to drive mobiles\n\nMobile pho...
6,./dataset/test/sport(1).txt,Newry to fight cup exit in courts\n\nNewry Cit...
7,./dataset/test/entertainment(1).txt,Rapper Snoop Dogg sued for 'rape'\n\nUS rapper...
8,./dataset/test/tech (12).txt,Apple makes blogs reveal sources\n\nApple has ...
9,./dataset/test/sport(8).txt,Hansen 'delays return until 2006'\n\nBritish t...


In [117]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

test_features = tfidf.transform(dataset_test.Content).toarray()
model_pipe = [random_forest,logistic_regression,multinomial_naive_bayes]
for model in model_pipe:
    predictions = model.predict(test_features)
    
    prediction_list = []
    for predicted in (predictions):
        type_file = id_to_category[predicted]
        type_file = type_file.split('/')[-1]
        prediction_list.append(type_file)

    OriginalType = []
    for i in dataset_test['File Names']:
        file_name = i.split('/')[-1]
        file_type = file_name.split('(')[0]
        OriginalType.append(file_type)

    encoder = LabelEncoder()
    prediction_list = encoder.fit_transform(prediction_list)
    OriginalType = encoder.fit_transform(OriginalType)
    
    accuracy = accuracy_score(prediction_list, OriginalType)

    print("Accuracy is {} for {}".format(accuracy,model))

# dataset_test.to_excel("output_document_segmentation.xlsx")

Accuracy is 0.7391304347826086 for RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
Accuracy is 0.8695652173913043 for LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Accuracy is 0.9565217391304348 for MultinomialNB(alpha=1.0, class_

In [118]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

test_features = tfidf.transform(dataset_test.Content).toarray()
predictions = multinomial_naive_bayes.predict(test_features)

prediction_list = []
for predicted in (predictions):
    type_file = id_to_category[predicted]
    type_file = type_file.split('/')[-1]
    prediction_list.append(type_file)

dataset_test['Type'] = prediction_list

OriginalType = []
for i in dataset_test['File Names']:
    file_name = i.split('/')[-1]
    file_type = file_name.split('(')[0]
    OriginalType.append(file_type)

dataset_test['OriginalType'] = OriginalType

In [119]:
import random
file_number = str(random.randint(1,100))
target_folder = "./sorted_files_" + file_number + "/"
required_folders = dataset_test['Type'].unique().tolist()
import os
for i in required_folders:
# define the name of the directory to be created
    path = target_folder + i
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s" % path)

for i in range(len(dataset_test)):
    print (i)
    source = dataset_test['File Names'][i]
    destination =  target_folder + "" + dataset_test['Type'][i] 
    shutil.copy(source, destination)

Successfully created the directory ./sorted_files_16/politics
Successfully created the directory ./sorted_files_16/tech
Successfully created the directory ./sorted_files_16/business
Successfully created the directory ./sorted_files_16/sport
Successfully created the directory ./sorted_files_16/entertainment
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
